In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_path = "/kaggle/input/titanic/train.csv"
test_path = "/kaggle/input/titanic/test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
men = train_data.loc[lambda train_data: train_data["Sex"] == "male"]["Survived"]


In [4]:
num_men = len(men)
perc_men = sum(men)/num_men * 100
print("Percentage of survivors who were men: {}%".format(round(perc_men, 3)))

Percentage of survivors who were men: 18.891%


In [42]:
# Begin building the model
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

features = ["Pclass", "Sex", "Age", "Fare"]
X_train = pd.get_dummies(train_data[features])
Y_train = train_data["Survived"]

mean_age = X_train["Sex_male"].mean()
X_train.loc[X_train['Age'].isnull(), 'Age'] = mean_age
X_train

X_test = pd.get_dummies(test_data[features])
mean_age_test = X_test["Sex_male"].mean()

# Handling null values in age
X_test.loc[X_test['Age'].isnull(), 'Age'] = mean_age_test

# Handling null values in fare
fare_mean = X_test.loc[X_test["Pclass"] == 3, "Fare"].mean()
X_test.loc[X_test['Fare'].isnull(), "Fare"] = round(fare_mean, 4)

imputer = SimpleImputer()
# new_test = imputer.fit_transform(X_test)



model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)

In [30]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': Y_pred})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

,Pclass,Age,Fare,Sex_female,Sex_male
0,3,34.5,7.8292,0,1
1,3,47.0,7.0000,1,0
2,2,62.0,9.6875,0,1
3,3,27.0,8.6625,0,1
4,3,22.0,12.2875,1,0
